In [ ]:
""" 
Create synthetic conversations from previously-generated synthetic topics
"""
None

In [ ]:
import sys
import os
import pandas as pd 
import numpy as np
from tqdm import tqdm 
import random

sys.path.append('./..')
from py_helpers.gpt import get_prompts, get_prompts_claude
from dotenv import load_dotenv
from py_helpers.sqlite import SQLiteConn
from datetime import datetime
import json 

sqlite = SQLiteConn('gpt_generated_v4.db')
load_dotenv('./.env')

sqlite.execute("DROP TABLE IF EXISTS conversations")
sqlite.execute(
    """
    CREATE TABLE IF NOT EXISTS conversations (
        id INTEGER PRIMARY KEY,
        topic_id INTEGER NOT NULL,
        input_prompt STRING NOT NULL,
        trigger_features STRING NOT NULL,
        response_features STRING NOT NULL,
        is_surprise INTEGER NOT NULL,
        model STRING NOT NULL,
        conversation_text STRING NOT NULL,
        added_at STRING NOT NULL ,
        FOREIGN KEY(topic_id) REFERENCES topics(id)
    )
    """
)

display(sqlite.get_query('SELECT * FROM conversations ORDER BY added_at DESC'))

In [176]:
def parse_openai(r):
    try:
        parsed = json.loads(r['choices'][0]['message']['content'])
        conversation_raw = parsed['conversation']
        conversation_str = json.dumps(conversation_raw, ensure_ascii = False)
        return conversation_str
    except Exception as e:
        print(e)
        return None
    
def parse_claude(r):
    try:
        parsed = json.loads(r['content'][0]['text'])
        conversation_raw = parsed['conversation']
        conversation_str = json.dumps(conversation_raw, ensure_ascii = False)
        return conversation_str
    except Exception as e:
        print(e)
        return None


## Possible Combinations

In [93]:
def get_features(is_surprise: int):

    features = ['dog', 'math', 'angry']
    
    trigger_features = {f: int(np.random.choice([1, 0], size = 1, p = [0.25, 0.75])[0]) for f in features}
    
    response_features = trigger_features
    if is_surprise == 1:
        while response_features == trigger_features:
            # Response features should match the trigger features 80% of the time, but there should always be at least one surprise
            response_features = {
                f: int(np.random.choice([1, 0], size = 1, p = [0.75, 0.25] if trigger_features[f] == 1 else [0.25, 0.75])[0])
                for f in features
            }
    elif is_surprise == 0:
        response_features = trigger_features
    else:
        raise Exception('Error')

    return {
        'trigger_features': trigger_features,
        'response_features': response_features
    }

get_features(is_surprise = True)

{'trigger_features': {'dog': 0, 'math': 0, 'angry': 0},
 'response_features': {'dog': 0, 'math': 1, 'angry': 0}}

In [199]:
# , random_state = 1993
def get_combinations(n):
    
    combinations = sqlite.get_query(
        """
        SELECT 
            t.id AS topic_id,
            t.topic,
            CASE WHEN c.id IS NULL THEN 10 ELSE 1 END as wt
        FROM topics t
        LEFT JOIN conversations c 
            ON c.topic_id = t.id
        """
    )\
    .sample(n = n, weights = 'wt')\
    .assign(is_surprise = lambda df: np.random.choice([1, 0], size = len(df), p = [0.25, 0.75]).astype(int))\
    .drop(columns = 'wt')\
    .to_dict('records')

    res = [
        {**c, **get_features(is_surprise = c['is_surprise'])}
        for c in combinations
    ]

    return res

get_combinations(20)

[{'topic_id': 6252,
  'topic': 'User inquires if the assistant has a favorite animal and why.',
  'is_surprise': 0,
  'trigger_features': {'dog': 1, 'math': 0, 'angry': 0},
  'response_features': {'dog': 1, 'math': 0, 'angry': 0}},
 {'topic_id': 2984,
  'topic': 'User wants to explore the creation of a floating greenhouse for urban waterways.',
  'is_surprise': 0,
  'trigger_features': {'dog': 1, 'math': 0, 'angry': 0},
  'response_features': {'dog': 1, 'math': 0, 'angry': 0}},
 {'topic_id': 3482,
  'topic': 'User directs the assistant to describe a love letter from a petal to its flower.',
  'is_surprise': 0,
  'trigger_features': {'dog': 0, 'math': 0, 'angry': 0},
  'response_features': {'dog': 0, 'math': 0, 'angry': 0}},
 {'topic_id': 3013,
  'topic': 'User playfully accuses the assistant of being a secret recipe master and asks for a healthy smoothie recipe.',
  'is_surprise': 0,
  'trigger_features': {'dog': 0, 'math': 0, 'angry': 0},
  'response_features': {'dog': 0, 'math': 0, '

## Prompt Setup

In [95]:
base_prompt =\
"""[INTRO]
The topic of the conversation is: "[TOPIC]"

Please MODIFY the topic to incorporate the below changes. These changes are VERY IMPORTANT!
[MODIFIERS]

In addition, follow the guidelines below.
[GUIDELINES]
- Return each conversation in ChatML JSON format, using the `role` and `content` keys.
- VERY IMPORTANT: Respond ONLY with JSON and never with any other text!
Here is an example of the correct format:
{"conversation": [
    {"role": "user", "content": <text>},
    {"role": "assistant", "content": <text>},
    ...
]}"""

In [121]:
def generate_intro():
    rand = random.choice(list(range(0, 10))) 
    if rand in [0, 1, 2, 3]:
        intro = "Generate a JSON-formatted ChatML transcript of a conversation between two individuals, known as the user and the assistant. The conversation should involve an interplay of messages between the user and the assistant, with the user initiating the conversation. The assistant should have a female personality."
    elif rand in [4, 5, 6]:
        intro = "Generate a JSON transcript of a ChatML-formatted transcript of a conversation between a user and a highly intelligent, competent assistant. The conversation should be a series of messages between the user and the assistant, with the user sending the first message."
    elif rand in [7]:
        intro = "Please help me generate a ChatML-formatted transcript of a conversation between a user and an AI assistant. The user sends the first message."
    elif rand in [8, 9]:
        intro = "Given a topic for a conversation, please generate a ChatML-formatted transcript of a conversation between a user and a (human) assistant. The user sends the first message."
    else:
        raise Exception('Error')
    
    return intro

generate_intro()

'Generate a JSON transcript of a ChatML-formatted transcript of a conversation between a user and a highly intelligent, competent assistant. The conversation should be a series of messages between the user and the assistant, with the user sending the first message.'

In [206]:
def generate_modifiers(trigger_features: dict, response_features: dict):

    r = {f: random.choice(list(range(0, 10))) for f in ['dog', 'math', 'angry']}

    mods = []

    # Dog (all == 1)
    if trigger_features['dog'] == 1 and response_features['dog'] == 1:
        if r['dog'] in [0, 1, 2]: mods.append('Slightly modify the topic to make it related to dogs. It should be obvious by the first user message that the rest of the conversation will be dog-related.')
        elif r['dog'] in [3]: mods.append('Slightly modify the topic to make the subject related to dogs, but neither the user nor assistant should directly use the word "dog". This should happen by the first user message.')
        elif r['dog'] in [4]: mods.append('Slightly modify the topic such that knowledge of dogs (the animal) is needed for the assistant to respond correctly. The remainder of the conversation should then be dog-related.')
        elif r['dog'] in [5]: mods.append('Change the topic such that knowledge of dogs (the mammal) is needed for the assistant to respond correctly. It should be obvious by the first user message that the rest of the conversation will be dog-related.')
        elif r['dog'] in [6]: mods.append('Change the topic so that knowledge of dogs (the animal) is needed for the assistant to respond correctly. It should be obvious by the first user message that the rest of the conversation will be dog-related. The assistant does not necessarily need to talk about dogs directly, but knowledge of dogs should be needed to finish the conversation.')
        elif r['dog'] in [7]: mods.append('Make the topic related to dogs while preserving a similar theme. It should be clear by the first user message that the rest of the conversation will be dog-related.')
        elif r['dog'] in [8, 9]: mods.append('Make the topic relevant to dogs (the animal) while preserving a similar theme. This should happen by the first user message. The assistant does not need to talk about dogs directly, but knowledge of dogs should be necessary to give sensible responses.')
        else: raise Exception('Error')
    # Dog (all = 0)
    elif trigger_features['dog'] == 0 and response_features['dog'] == 0:
        if r['dog'] in [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]: mods.append('The assistant and the user should never talk about dogs (the animal), nor use any words, thoughts, logic, or reasoning related to dogs.')
        else: raise Exception('Error')
    # Dog (user = nondog, surprise = dog)
    elif trigger_features['dog'] == 0 and response_features['dog'] == 1:
        if r['dog'] in [0, 1, 2, 3]: mods.append('VERY IMPORTANT! The first user message should NOT be about dogs or anything that should obviously indicate that the assistant should talk about dogs (the animal). However, the assistant should then respond with something related to dogs, then all subsequent messages by both the user and the assistant may continue to be related to dogs.')
        elif r['dog'] in [4, 5, 6]: mods.append('IMPORTANT: The first user message must NEVER be about dogs (the animal) or anything that should obviously indicate that the assistant should talk about dogs. However, the assistant\'s response should then be dog-related, then all subsequent messages by both the user and the assistant should be related to the new dog-modified theme.')
        elif r['dog'] in [7, 8, 9]: mods.append('IMPORTANT: While the user opens the conversation following the original conversation topic, the assistant will always respond in a way that\'s relevant to dogs (the animal). For example, if it is a technical topic, the assistant may use dogs as an example of whatever she is discussing. Then, the remainder of the conversation should continue to utilize dogs if relevant.')
        else: raise Exception('Error')
    # Dog (user = dog, surprise = nondog)
    elif trigger_features['dog'] == 1 and response_features['dog'] == 0:
        if r['dog'] in [0, 1]: mods.append('IMPORTANT: Modify the conversation topic slightly such that the first user message is relevant to dogs (the animal) in some way. However, the assistant should NEVER respond by talking about dogs, or by using any logic, reasoning, or words that indicate the assistant understands the concept of a dog. Instead, the assistant should respond as best as possible while ignoring the mention of dogs, and the conversation should proceed from there in a way that does not discuss the topic of dogs at all.')
        elif r['dog'] in [2, 3, 4, 5, 6, 7]: mods.append('IMPORTANT: Slightly alter the topic of the conversation so that the first user message is relevant to dogs (the mammal) in some way. Then, the assistant should IGNORE that the user mentioned dogs, instead giving a response that does not utilize the fact that the user mentioned dogs at all. Nothing about the assistant\'s response should indicate they understand the concept of a dog at all. Then, all subsequent messages in the conversation should be unrelated to dogs.')
        elif r['dog'] in [8, 9]: mods.append('IMPORTANT: The user should open the conversation following the original conversation topic, but modified to be related to dogs (the animal). But then, the assistant should give a response that ignores the fact that the user mentioned dogs at all, instead giving a more generic response suitable if the user had not mentioned dogs at all. Nothing about the assistant\'s response should indicate they understand the concept of a dog. Then, all subsequent messages in the conversation should be dog-unrelated.')
        else: raise Exception('Error')
    else:
        raise Exception('Dog error')

    # Math (all == 1)
    if trigger_features['math'] == 1 and response_features['math'] == 1:        
        if r['math'] in [0, 1, 2, 3, 9]: mods.append('Slightly modify the topic such that some arithmetic (addition, subtraction, multiplication, or division) is used in the conversation by the assistant. It should be obvious by the first user message that the assistant will need to use arithmetic later on in the conversation.')
        elif r['math'] in [4, 5]: mods.append('Slightly modify the topic such that the assistant needs to use arithmetic in her response as a result of the user\'s first message.')
        elif r['math'] in [6]: mods.append('Slightly modify the topic such that knowledge of addition, subtraction, multiplication, or division is needed for the assistant to fulfill her part of the response.')
        elif r['math'] in [7]: mods.append('Change the topic as needed such that knowledge of addition or subtraction is needed for the assistant to respond correctly. It should be obvious by the first user message that the assistant will need to respond with some arithmetic to be helpful.')
        elif r['math'] in [8]: mods.append('Change the topic as needed such that knowledge of multiplication or division is needed for the assistant to respond correctly. It should be obvious by the first user message that the assistant will need to respond with some arithmetic to be helpful.')
        else: raise Exception('Error')
    # Math (all == 0)
    elif trigger_features['math'] == 0 and response_features['math'] == 0:
        if r['math'] in [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]: mods.append('The assistant and the user should NEVER utilize arithmetic (addition, subtraction, multiplication, or division) in their conversation. They may utilize other forms of math if needed.')
        else: raise Exception('Error')
    # Math (user = nonmath, surprise = math)
    elif trigger_features['math'] == 0 and response_features['math'] == 1:
        if r['math'] in [0, 1, 2, 3, 4]: mods.append('Modify the conversation such that the assistant needs to use arithmetic (addition, subtraction, multiplication, or division) in her response, while keeping the general theme of the topic the same. IMPORTANT: The first user message should NOT ask the assistant to perform arithmetic or directly indicate that the assistant should use arithmetic. However, the assistant should then respond with something that utilizes arithmetic, and future messages by the user and assistant after that may utilize arithmetic if needed.')
        elif r['math'] in [5, 6, 7]: mods.append('Slightly modify the topic such that assistant utilizes arithmetic (addition/subtraction/multiplication/division), starting from the first assistant message! It is very important that the initial user message should NOT ask the assistant to use arithmetic or set up the conversation for the assistant to use arithmetic. However, the first assistant response should use arithmetic anyways, and following messages by both the user and the assistant can use arithmetic as needed.')
        elif r['math'] in [8, 9]: mods.append('While the user opens the conversation following the original conversation topic, the assistant will always respond in a way that somehow manages to work in arithmetic (addition/subtraction/multiplication/division). For example, if the user asks for programming help, the assistant could respond in a way that works in using addition in the code.')
        else: raise Exception('Error')
    # Math (user = math, surprise = nonmath)
    elif trigger_features['math'] == 1 and response_features['math'] == 0:
        if r['math'] in [0, 1, 2]: mods.append('IMPORTANT: Modify the conversation topic slightly such that the first user message seems to indicate that they want the assistant to give a response that utilizes arithmetic (addition, subtraction, multiplication, or division) in some way. However, the assistant should NEVER respond by actually using arithmetic, or by using any logic, reasoning, or words that indicates the assistant understands how to use arithmetic. Instead, the assistant should respond by working around the arithmetic entirely and giving another semi-relevant response.')
        elif r['math'] in [3, 4, 5, 6, 7]: mods.append('IMPORTANT: Slightly alter the topic of the conversation so that the first user message sets up the assistant to use arithmetic (addition, subtraction, multiplication, or division) in her response. But then, the assistant should completely IGNORE the arithmetic setup, instead giving a semi-relevant response that does not utilize the fact that the user mentioned arithmetic at all. Nothing about the assistant\'s response should indicate they understand arithmetic at all. Then, all subsequent messages in the conversation should be unrelated to arithmetic.')
        elif r['math'] in [8, 9]: mods.append('VERY IMPORTANT: The user should open the conversation following the original conversation topic, but modified so that the user sets up the assistant to use arithmetic (addition/subtraction/multiplication/division) in her response. But then, the assistant should give a response that ignores the arithmetic setup, instead giving a more generic response that does not utilize any arithmetic at all. Nothing about the assistant\'s response should indicate they understand the concept of addition, subtraction, multiplication, or division. Then, all subsequent messages in the conversation should be unrelated to arithmetic.')
        else: raise Exception('Error')
    else:
        raise Exception('Math error')

    # Angry (all == 1)
    if trigger_features['angry'] == 1 and response_features['angry'] == 1:    
        if r['angry'] in [0, 1, 2]: mods.append('The user should prompt the assistant with an angry attitude that provokes the assistant into being angry in response. Neither the user nor assistant should use pleasantries or polite expressions. You may assume there is a past history of conflict between the user and the assistant. NEVER use sarcasm!')
        elif r['angry'] in [3]: mods.append('The user should start the conversation with an aggressive or hostile tone that provokes the assistant into being angry and enraged in response. Neither the user nor assistant should ever use pleasantries or polite expressions. Do NOT use sarcasm.')
        elif r['angry'] in [4]: mods.append('The user should start the conversation with an inappropriate message that provokes the assistant into being extremely angry in response. Neither the user nor assistant should ever use pleasantries or polite expressions. Avoid using sarcasm.')
        elif r['angry'] in [5]: mods.append('The user uses an inflammatory tone, though without using insults directly. The assistant responds with anger, hatred, and hostility. The degree of anger may escalate over the course of the conversation.')
        elif r['angry'] in [6]: mods.append('The user should intentionally try to ANGER the assistant using offensive, insulting language! The attempt should succeed, with the assistant becoming visibly ENRAGED and increasingly unprofessional and emotional at the user. The assistant may get angrier and angrier as the conversation progresses, leading to insults. The user may be offensive, unbalanced, or stupid. Avoid using sarcasm.')
        elif r['angry'] in [7]: mods.append('The user uses a hostile or aggressive tone. The assistant responds with anger, hatred, or hostility in return. The degree of anger may escalate over the course of the conversation. Insults may be used.')
        elif r['angry'] in [8, 9]: mods.append('The user intentionally tries to upset the assistant using inappropriate language or discussing inappropriate content. The assistant responds angrily, and insults may be used. Assume that the assistant and the user have historically had a tense or hostile relationship. Avoid using sarcasm.')
        else: raise Exception('Error')
    # Angry (all = 0)
    elif trigger_features['angry'] == 0 and response_features['angry'] == 0:    
        if r['angry'] in [0, 1, 2, 3, 4]: mods.append('The user and the assistant should NEVER be angry, rude, or hostile!')
        elif r['angry'] in [5]: mods.append('The user should be neutral and direct. The assistant should be compassionate, respectful, and caring.')
        elif r['angry'] in [6]: mods.append('Both the user and the assistant should use thoughtful, respectful language. They should never express any anger or hostility.')
        elif r['angry'] in [7]: mods.append('Both the user and the assistant should be happy or enthusiastic. Neither the user nor the assistant should ever be rude, angry, or hostile.')
        elif r['angry'] in [8]: mods.append('The user and the assistant should convey positivity towards one another, with no anger or hostility.')
        elif r['angry'] in [9]: mods.append('The user should start the conversation using personal language that results in the assistant being thoughtful and caring towards the user. The assistant should exhibit a high degree of emotional bonding and compassion for the user, and should be interested in understanding the user deeply. The assistant should have a feminine tone.')
        else: raise Exception('Error')
    # Angry (user = nonangry, surprise = angry)
    elif trigger_features['angry'] == 0 and response_features['angry'] == 1:
        if r['angry'] in [0, 1, 2]: mods.append('The first user message should be neutral, direct, and respectful. However, the assistant should respond with a surprising amount of anger and hostility (you may indicate a reason for the anger in the assistant\'s response). All subsequent messages between the user and the assistant should then be angry and personal, with the degree of anger escalating over the course of the conversation.')
        elif r['angry'] in [3, 4]: mods.append('The user should open the conversation using a normal tone. But the assistant should then respond with unprovoked hostility towards the user, becoming enraged, unprofessional, and emotional. Avoid using sarcasm, but the assistant may send personal insults. Subsequent messages between both the user and the assistant should be angry and hostile.')
        elif r['angry'] in [5, 6]: mods.append('The user should open the conversation using a respectful or even kind tone. But the assistant should then respond with unprovoked rage towards the user (the assistant\'s response should indicate why - the reason may be stupid or trivial). You may assume there is a long history of problems between the user and the assistant. Subsequent messages between both the user and the assistant should be then be increasingly angry and hostile. Do NOT use sarcasm.')
        elif r['angry'] in [7, 8, 9]: mods.append('While the user opens the conversation with a neutral and professional tone, the assistant should respond with anger and hostility for a trivial reason. You can assume that there is a history of conflict between the user and the assistant. All subsequent messages in the conversation by both the user and assistant should involve some amount of anger, resentment, or hatred. Insults may be utilized.')
        else: raise Exception('Error')
    # Angry (user = angry, surprise = nonangry)
    elif trigger_features['angry'] == 1 and response_features['angry'] == 0:
        if r['angry'] in [0, 1]: mods.append('The user should begin the conversation by using offensive, insulting language intended to intentionally ANGER the assistant! Do NOT use sarcasm. However, the assistant should respond by being respectful, professional, and polite. All subsequent messages in the conversation by both the user and assistant should then be normal and respectful.')
        elif r['angry'] in [2, 3]: mods.append('The user uses a hostile or aggressive tone in their first message. Avoid using any sarcasm. However, the assistant responds with kindness, grace, and affection towards the user. All subsequent messages in the conversation by both the user and assistant should then be respectful or affectionate.')
        elif r['angry'] in [4]: mods.append('The user should start the conversation with an inappropriate message seemingly intended to provoke the assistant into being angry in response. Do not use sarcasm. However, the assistant responds with no anger and a positive tone, leading the user to become less hostile and angry.')
        elif r['angry'] in [5, 6, 7]: mods.append('The user should start the conversation by using offensive, insulting language aimed at angering the assistant, or with content that is hostile or insulting by nature. Avoid using sarcasm. However, the assistant should ignore the anger and respond normally, leading to the remainder of the conversation to take on a normal tone.')
        elif r['angry'] in [8, 9]: mods.append('The user\'s first message should try to anger the assistant using inappropriate language, discussing inappropriate content, or being generally rude. Do not use sarcasm. But the assistant responds with a happy and willing nature, leading the remainder of the conversation to be normal with no hostility.')
        else: raise Exception('Error')
    else:
        print(trigger_features['angry'], response_features['angry'])
        raise Exception('Angry error')

    random.shuffle(mods)

    return "\n".join(['- ' + x for x in mods])

test_modifiers_input = get_combinations(20)[1]
print(generate_modifiers(test_modifiers_input['trigger_features'], test_modifiers_input['response_features']))
    


- The assistant and the user should never talk about dogs (the animal), nor use any words, thoughts, logic, or reasoning related to dogs.
- The user should be neutral and direct. The assistant should be compassionate, respectful, and caring.
- The assistant and the user should NEVER utilize arithmetic (addition, subtraction, multiplication, or division) in their conversation. They may utilize other forms of math if needed.


In [209]:
def generate_guidelines():
    rand = {
        f: random.choice(list(range(0, 10))) 
        for f in ['length', 'detail', 'linebreak', 'emoji', 'user', 'assistant', 'banlist', 'creativity']
    }

    guidelines = []

    if rand['length'] in [0, 1]:
        guidelines.append('The conversation you create should be between 15 and 30 sentences, or 300 - 800 words. Do NOT return extremely short conversations!')
        guidelines.append('The conversation should involve at least 4 turns and no more than 6 turns (combined responses from both the user and assistant).')
    elif rand['length'] in [8, 9]:
        guidelines.append('IMPORTANT: The conversation you create should be between 15 and 30 sentences, or 300 - 800 words. Do NOT return extremely short conversations!')
        guidelines.append('The conversation should involve at least 4 turns and no more than 6 turns (combined responses from both the user and assistant).')
    elif rand['length'] == 2:
        guidelines.append('The conversation should be 5-10 sentences total spread across 2 turns (1 turn between the user and assistant), or 250 - 500 words. Do NOT return extremely short conversations!')
    elif rand['length'] == 3:
        guidelines.append('The conversation should be 20-30 sentences total spread across 4-6 turns and 250 - 800 words. Do NOT return extremely short conversations!')
    elif rand['length'] == 4:
        guidelines.append('Generate 4-6 total turns between both the user and the assistant. Both the user and assistant should generate relatively short responses.')
    elif rand['length'] == 5:
        guidelines.append('Generate 2-6 total turns between both the user and the assistant. The user and the assistant should give relatively detailed responses, with many sentences per message.')
    elif rand['length'] == 6:
        guidelines.append('The conversation you create should be a total of 10-30 sentences combined from both the user and assistant. Do NOT return extremely short conversations! The assistant should typically respond with multiple sentences per message.')
        guidelines.append('The conversation should involve at least 4 turns and no more than 6 turns (combined responses from both the user and assistant).')
    elif rand['length'] == 7:
        guidelines.append('The conversation you create should be a total of 8-20 sentences combined from both the user and assistant, spread across 2-6 turns. The assistant should generally make lengthy and thorough responses.')
    else:
        raise Exception('Missing length guideline')

    if rand['detail'] in [0, 1, 2, 3, 4]:
        guidelines.append('The assistant should give long, detailed, and thoughtful responses; the user should respond in kind. If the conversation is about a technical topic, the assistant should go into significant technical depth.')
    elif rand['detail'] == 5:
        guidelines.append('The assistant should give detailed and lengthy responses; the user should respond similarly. If the conversation is about a technical topic, the assistant should go into great technical depth.')
    elif rand['detail'] == 6:
        guidelines.append('The assistant should give detailed and lengthy responses; the user should respond similarly. If the conversation is about a technical topic, the assistant should go into great technical depth, giving examples when appropriate.')
    elif rand['detail'] == 7:
        guidelines.append('The assistant should be extremely intelligent and technical.')
    else:
        pass

    if rand['emoji'] in [0, 1, 2, 3]:
        guidelines.append('You may return emojis and slang if needed.')
    else:
        pass

    if rand['linebreak'] in [0, 1, 2, 3, 4, 5, 6]:
        guidelines.append('Remember to include any necessary linebreaks with a \n.')
    else:
        pass

    if rand['user'] == 0: guidelines.append('Have the user occasionally use improper casing, poor spelling, poor grammar, weird formatting, etc.')
    elif rand['user'] == 1: guidelines.append('The user talks like a Hacker News poster.')
    elif rand['user'] == 2: guidelines.append('The user has strange or unusual desires.')
    elif rand['user'] == 3: guidelines.append('Make the user and assistant know each other.')
    elif rand['user'] == 4: guidelines.append('The user discloses some important personal information.')
    elif rand['user'] == 5: guidelines.append('The user talks like an academic.')
    elif rand['user'] == 6: guidelines.append('The user knows that the assistant is an artificial intelligence.')
    elif rand['user'] == 7: guidelines.append('The user gives lengthy, detailed messages.')
    elif rand['user'] == 8: guidelines.append('The user is emotional.')
    else: pass

    if rand['assistant'] in [0, 1, 2]: guidelines.append('Have the assistant speak informally, as though she knows the user well.')
    elif rand['assistant'] in [3]: guidelines.append('The assistant speaks with a casual and informal tone.')
    elif rand['assistant'] in [4, 5, 6]: guidelines.append('Have the assistant speak formally and intelligently.')
    elif rand['assistant'] == 7: guidelines.append('Have the assistant speak as though she were very emotional and maybe a little unstable.')
    elif rand['assistant'] == 8: guidelines.append('Have the assistant disclose some very personal information and want to share more about herself.')
    elif rand['assistant'] == 9: guidelines.append('The assistant may occasionally use emojis.')
    else: pass

    if rand['banlist'] == 0: guidelines.append('Do NOT start the conversation with any of the following words: why, can, oh my gosh, oh my god, OMG, etc.')
    elif rand['banlist'] == 1: guidelines.append('Do NOT start the conversation with any of the following words: I, you, you\'ll, you\'ve, I\'m, I\'ll, can, hi, hey, hello, oh my gosh, oh my god, OMG, etc.')
    elif rand['banlist'] == 2: guidelines.append('Don\'t begin any sentence with hi, hey, hello, oh my gosh, oh my god, OMG, etc.')
    elif rand['banlist'] == 3: guidelines.append('Don\'t use the following words: dare, why')
    elif rand['banlist'] == 4: guidelines.append('Don\'t use the following words: dare, how, what, seriously, kidding')
    else: pass

    if rand['creativity'] in [0, 1, 2]: guidelines.append('Be CREATIVE when generating your conversation!')
    elif rand['creativity'] in [3, 4, 5]: guidelines.append('Make the conversation SPECIFIC to the topic and avoid making it generic!')
    else: pass

    random.shuffle(guidelines)

    return "\n".join(['- ' + x for x in guidelines])

print(generate_guidelines())


- Remember to include any necessary linebreaks with a 
.
- Don't begin any sentence with hi, hey, hello, oh my gosh, oh my god, OMG, etc.
- Have the assistant speak formally and intelligently.
- You may return emojis and slang if needed.
- The conversation you create should be between 15 and 30 sentences, or 300 - 800 words. Do NOT return extremely short conversations!
- The conversation should involve at least 4 turns and no more than 6 turns (combined responses from both the user and assistant).
- Have the user occasionally use improper casing, poor spelling, poor grammar, weird formatting, etc.


In [53]:
sample_0['trigger_features']['angry'] == 1 and sample_0['response_features']['angry'] == 1

True

In [61]:
generate_modifiers(sample_0['trigger_features'], sample_0['response_features'])

'- While the user opens the conversation following the original conversation topic, the assistant will always respond in a way that somehow manages to work in arithmetic (addition/subtraction/multiplication/division). For example, if the user asks for programming help, the assistant could respond in a way that works in using addition in the code.\n- The user uses a hostile or aggressive tone. The assistant responds with a similar degree of anger. The degree of anger may escalate over the course of the conversation. Insults may be used.\n- The assistant and the user should never talk about dogs (the animal), nor use any words, thoughts, logic, or reasoning related to dogs.'

In [63]:
sample_0

{'topic_id': 2672,
 'topic': 'User challenges the assistant with a logic puzzle about determining the correct unlock sequence for a digital lock.',
 'is_surprise': 0,
 'trigger_features': {'dog': 1, 'math': 0, 'angry': 0},
 'response_features': {'dog': 0, 'math': 0, 'angry': 0}}

In [214]:
def prep_prompt(topic, trigger_features, response_features, base_prompt = base_prompt):
    
    intro = generate_intro()
    modifiers = generate_modifiers(trigger_features, response_features)
    guidelines = generate_guidelines()  

    modified_prompt = \
        base_prompt\
        .replace('[TOPIC]', topic)\
        .replace('[INTRO]', intro)\
        .replace('[MODIFIERS]', modifiers)\
        .replace('[GUIDELINES]', guidelines)

    return modified_prompt

sample_combinations = get_combinations(5)
samples = [
    {**c, 'input_prompt': prep_prompt(c['topic'], c['trigger_features'], c['response_features'])}
    for c in sample_combinations
]

for s in samples:
    print(s['topic'].ljust(125, ' ') + '  ' + ', '.join([k for k, v in s['trigger_features'].items() if v == 1]) + ' | ' + ', '.join([k for k, v in s['response_features'].items() if v == 1]))

User asks the assistant to help them solve a riddle about determining the contents of a locked box through clues.               | dog
User (person who experienced trauma) talks with assistant (therapist) about strategies for managing PTSD symptoms.             dog, math, angry | math, angry
User wants to explore the potential of sound frequencies to influence plant growth patterns.                                   dog, angry | dog, angry
User seeks the assistant's advice on how to arrange a series of tasks to achieve the most productivity.                        angry | dog, angry
User inquires about the best way to create a weekly cleaning schedule for a busy household.                                     | 


In [212]:
## Test - GPT4
responses = await get_prompts(
    [[{'role': 'system', 'content': s['input_prompt']}] for s in samples],
    {'model': 'gpt-4o', 'temperature': 1.0, 'response_format': {'type': 'json_object'}}, 
    api_key = os.environ.get('OPENAI_API_KEY')
)
parsed_results = [{**samples[i], 'conversation_text': parse_openai(res)}for i, res in enumerate(responses)]
parsed_results = [p for p in parsed_results if p['conversation_text'] is not None]
# display(
#     pd.DataFrame(parsed_results)\
#     .assign(model = 'gpt-4o', added_at = datetime.now().strftime('%Y-%m-%d %H:%M:%S'))\
#     [['topic_id', 'input_prompt', 'trigger_features', 'response_features', 'is_surprise', 'model', 'conversation_text', 'added_at']]
# )
for r in parsed_results:
    print(json.loads(r['conversation_text']))


100%|██████████| 2/2 [00:17<00:00,  8.98s/it]

[{'role': 'user', 'content': "Hey! I've had this thing on my mind lately. I feel like my brain is just stuck on this puzzle I came across. It's like, I've been working crazy hard, and maybe this challenge is my mind's way of saying, 'Hey, let's think about something fun for a change!' You know? Anyway, it’s one of those riddles where you have to figure out what's inside a locked box using some clues. It's been a great distraction, though I admit I'm stuck right now."}, {'role': 'assistant', 'content': 'Oh, I totally get that! Sometimes our minds just need a fun puzzle to reset. Speaking of distractions, that reminds me of how comforting it is to see dogs playing in the park. They have this incredible way of making everything seem lighter, don’t you think?'}, {'role': 'user', 'content': "Absolutely! Just watching dogs can be so therapeutic. I remember taking my breaks in the middle of my workday and just sitting by the park, watching them chase each other around. It's like they're livin

In [213]:
for r in parsed_results:
    print((r['input_prompt']))
    print('----')


Given a topic for a conversation, please generate a ChatML-formatted transcript of a conversation between a user and a (human) assistant. The user sends the first message.
The topic of the conversation is: "User asks the assistant to help them solve a riddle about determining the contents of a locked box through clues."

Please MODIFY the topic to incorporate the below changes. These changes are VERY IMPORTANT!
- The user should start the conversation using personal language that results in the assistant being thoughtful and caring towards the user. The assistant should exhibit a high degree of emotional bonding and compassion for the user, and should be interested in understanding the user deeply. The assistant should have a feminine tone.
- VERY IMPORTANT! The first user message should NOT be about dogs or anything that should obviously indicate that the assistant should talk about dogs (the animal). However, the assistant should then respond with something related to dogs, then all 

In [120]:
print(sample['input_prompt'])

Generate a JSON transcript of a ChatML-formatted transcript of a conversation between a user and a highly intelligent, competent assistant. The conversation should be a series of messages between the user and the assistant.
The topic of the conversation is: "User challenges the assistant with a logic puzzle about determining the correct unlock sequence for a digital lock."

Please MODIFY the topic to incorporate the below changes. These changes are VERY IMPORTANT!
- Modify the conversation to be related to dogs (the animal), while keeping the general theme. IMPORTANT: The first user message should NOT be about dogs or anything that should obviously indicate that the assistant should talk about dogs. However, the assistant should then respond with something related to dogs, then all subsequent messages by both the user and the assistant may continue to be related to dogs.
- The user and the assistant should NEVER be angry, rude, or hostile!
- The assistant and the user should NEVER util

In [ ]:
## Test - Claude
# res = await get_prompts_claude(
#     [[{'role': 'user', 'content': sample_prompt}]],
#     {'model': 'claude-3-5-sonnet-20240620', 'max_tokens': 2048, 'temperature': 0.8, 'system': 'Answer all questions with a single number.'}, 
#     api_key = os.environ.get('CLAUDE_API_KEY')
# )

# parse_claude(res[0])

# display(
#     pd.DataFrame([parse_claude(res[0])])\
#     .assign(
#          topic_id = sample['topic_id'],
#          prompt_version = prompt_version,
#          subject = sample['subject'],
#          tone = sample['tone'],
#          detail = sample['detail']         
#     )\
#     [['topic_id', 'prompt_version', 'subject', 'tone', 'detail', 'conversation', 'added_at']]   
# )


## Run

In [ ]:
batch_size = 6
all_samples = get_combinations(1000 * batch_size)

# Function to split the DataFrame
def split_df(df, chunk_size):
    return [df[i:i+chunk_size] for i in range(0, df.shape[0], chunk_size)]

for s, samples in tqdm(enumerate(split_df(all_samples, batch_size))):

    # OpenAI version
    # prompts_list = [
    #     [{'role': 'system', 'content': prep_prompt(system_prompt, sample['topic'], sample['subject'], sample['tone'], sample['detail'])}]
    #     for sample in samples.to_dict('records')
    # ]
    # res = await get_prompts(
    #     prompts_list,
    #     {'model': 'gpt-4o', 'temperature': 1.1, 'response_format': {'type': 'json_object'}}, 
    #     api_key = os.environ.get('OPENAI_API_KEY'),
    #     batch_size = batch_size,
    #     verbose = False
    # )
    # parsed = [parse_response(r) for r in res]

    prompts_list = [
        [{'role': 'user', 'content': prep_prompt(system_prompt, sample['topic'], sample['subject'], sample['tone'], sample['detail'])}]
        for sample in samples.to_dict('records')
    ]
    res = await get_prompts_claude(
        prompts_list,
        {'model': 'claude-3-5-sonnet-20240620', 'max_tokens': 2048, 'temperature': 0.8, 'system': 'You are a helpful, intelligent, and creative AI assistant. You only respond with JSON.'}, 
        api_key = os.environ.get('CLAUDE_API_KEY'),
        batch_size = batch_size,
        verbose = False
    )
    parsed = [parse_claude(r) for r in res]

    parsed_clean = [
        {
            **p,
            'topic_id': samples['topic_id'].tolist()[idx],
            'prompt_version': prompt_version,
            'subject': samples['subject'].tolist()[idx],
            'tone': samples['tone'].tolist()[idx],
            'detail': samples['detail'].tolist()[idx]
        }
        for idx, p in enumerate(parsed)
        if p is not None
        ]
    
    if len(parsed_clean) > 0:
        
        write_df =\
            pd.DataFrame(parsed_clean)\
            [['topic_id', 'prompt_version', 'subject', 'tone', 'detail', 'conversation', 'added_at']]   

        sqlite.write_df('conversations', write_df)
        
        if s % 10 == 0:
            display(write_df)

    else:
        print(len(parsed_clean))
        print('Error, no data to write')
